In [ ]:
%%bash
if [[ ! -d "dataset" ]]; then
  mkdir dataset
  curl -L -o dataset/70-dog-breedsimage-data-set.zip\
    https://www.kaggle.com/api/v1/datasets/download/gpiosenka/70-dog-breedsimage-data-set
  unzip dataset/70-dog-breedsimage-data-set.zip -d dataset
fi

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import os
import copy
import time
from torch.optim import lr_scheduler
from collections import Counter

data_dir = 'dataset'
data_phases = ['train', 'valid']

# Definir transformaciones
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# Cargar los datasets usando ImageFolder
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in data_phases
}

# Crear DataLoaders
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in data_phases
}

# Información del dataset (importante para el modelo)
dataset_sizes = {x: len(image_datasets[x]) for x in data_phases}
class_names = image_datasets['train'].classes
num_classes = len(class_names)

# Configuración del dispositivo (CPU/GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. Obtener los índices de clase de todas las imágenes de entrenamiento
train_labels = [label for _, label in image_datasets['train'].samples]
class_counts = Counter(train_labels)
class_counts_list = [class_counts[i] for i in range(num_classes)]

# 2. Calcular los pesos inversos
total_samples = sum(class_counts_list)
weights_list = [total_samples / count for count in class_counts_list]
weights_tensor = torch.tensor(weights_list, dtype=torch.float32)

# Definir la función de pérdida con pesos
criterion = nn.CrossEntropyLoss(weight=weights_tensor.to(device))

print(f"Clases detectadas: {num_classes}")
print(f"Imágenes de Entrenamiento: {dataset_sizes['train']}")
print(f"Imágenes de Validación: {dataset_sizes['valid']}")

Clases detectadas: 70
Imágenes de Entrenamiento: 7946
Imágenes de Validación: 700


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# 1. Obtener los índices de clase de todas las imágenes de entrenamiento
train_labels = [label for _, label in image_datasets['train'].samples]
# 2. Contar las ocurrencias de cada clase
class_counts = Counter(train_labels)
# Asegurar que los conteos estén en el orden correcto de los índices de clase (0 a num_classes-1)
class_counts_list = [class_counts[i] for i in range(num_classes)]

# 3. Calcular los pesos inversos (pesos más altos para clases menos frecuentes)
# El peso es inversamente proporcional a la frecuencia de la clase
total_samples = sum(class_counts_list)
weights_list = [total_samples / count for count in class_counts_list]

# Normalizar los pesos (opcional, pero ayuda a la estabilidad)
# weights_tensor = torch.tensor(weights_list, dtype=torch.float32)
# weights_tensor = weights_tensor / weights_tensor.sum() * num_classes

weights_tensor = torch.tensor(weights_list, dtype=torch.float32)

print(f"Pesos de clases (ejemplo): {weights_tensor[:5]}")

Pesos de clases (ejemplo): tensor([ 75.6762,  72.8991,  67.9145, 122.2462,  89.2809])


In [ ]:
# Cargar el modelo ResNet18 pre-entrenado
model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Reemplazar la capa final (fully-connected)
num_ftrs = model_ft.fc.in_features
# La nueva capa de clasificación con Dropout
model_ft.fc = nn.Sequential(
    nn.Dropout(0.2), # Nueva capa de Dropout
    nn.Linear(num_ftrs, num_classes)
)

try:
    # --- PASO DE SOLUCIÓN DE ERRORES ---
    # 1. Cargar el state_dict directamente (no en el modelo aún)
    state_dict = torch.load('resnet18_70_breeds_best_weights_v2.pth')

    # 2. Renombrar las claves antiguas a las nuevas claves secuenciales
    # La clave 'fc.weight' debe ser 'fc.1.weight'
    # La clave 'fc.bias' debe ser 'fc.1.bias'

    # NOTA: Usamos .pop() para mover y eliminar las claves antiguas
    state_dict['fc.1.weight'] = state_dict.pop('fc.weight')
    state_dict['fc.1.bias'] = state_dict.pop('fc.bias')

    # 3. Cargar el state_dict modificado en el modelo
    model_ft.load_state_dict(state_dict)

    print("Pesos del mejor modelo (0.9171 Acc) cargados exitosamente después de corregir el nombre de las capas.")
except FileNotFoundError:
    print("ADVERTENCIA: Archivo de pesos no encontrado. Entrenando desde el inicio.")


# --- Bloque de Congelación Diferencial (se mantiene igual) ---

for name, param in model_ft.named_parameters():
    if name.startswith('fc'):
        # Siempre entrenamos la capa final
        param.requires_grad = True
    elif name.startswith('layer3') or name.startswith('layer4'):
        # Entrenamos las capas de alto nivel
        param.requires_grad = True
    else:
        # Congelamos conv1, bn1, layer1, layer2
        param.requires_grad = False

# Verificar cuántos parámetros se van a entrenar
params_to_update = [p for p in model_ft.parameters() if p.requires_grad]
print(f"Parámetros a entrenar: {len(params_to_update)} (Capas 3, 4 y FC)")

model_ft = model_ft.to(device)

# --- Bloque del Optimizador (se mantiene igual) ---

# Usamos un LR muy bajo, ya que el modelo ya está muy cerca de la solución.
optimizer_ft = optim.SGD(params_to_update, lr=0.00005, momentum=0.9) # LR ajustado a 0.00005

Pesos del mejor modelo (0.9171 Acc) cargados exitosamente después de corregir el nombre de las capas.
Parámetros a entrenar: 32 (Capas 3, 4 y FC)


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  # Iterar sobre las épocas
  for epoch in range(num_epochs):
      print(f'\nEpoch {epoch+1}/{num_epochs}')
      print('-' * 20)

      for phase in ['train', 'valid']:
          if phase == 'train':
              # El StepLR solía ir aquí. En ReduceLROnPlateau, va después de la validación.
              model.train()
          else:
              model.eval()

          running_loss = 0.0
          running_corrects = 0

          for inputs, labels in dataloaders[phase]:
              inputs = inputs.to(device)
              labels = labels.to(device)

              optimizer.zero_grad()

              with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / dataset_sizes[phase]
          epoch_acc = running_corrects.double() / dataset_sizes[phase]

          print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

          # **CAMBIO 5: Aplicar scheduler.step() después de la fase 'valid'**
          if phase == 'valid':
              # El scheduler se ajusta basado en la precisión de validación (epoch_acc)
              scheduler.step(epoch_acc)

              if epoch_acc > best_acc:
                  best_acc = epoch_acc
                  best_model_wts = copy.deepcopy(model.state_dict())

  time_elapsed = time.time() - since
  print(f'Entrenamiento completado en {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
  print(f'Mejor precisión de validación: {best_acc:.4f}')

  model.load_state_dict(best_model_wts)
  return model

In [ ]:
# ReduceLROnPlateau es un scheduler común para Fine-Tuning
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer_ft,
    mode='max',      # Observar la métrica que maximizamos (Acc)
    factor=0.1,      # Reducir el LR a 1/10
    patience=3      # Esperar 3 épocas sin mejora antes de reducir
)

# --- INICIO DEL ENTRENAMIENTO ---
NUM_EPOCHS = 50

model_ft = train_model(
    model_ft,
    criterion,
    optimizer_ft,
    exp_lr_scheduler,
    num_epochs=NUM_EPOCHS
)


Epoch 1/50
--------------------
train Loss: 0.9707 Acc: 0.7443
valid Loss: 0.7125 Acc: 0.9071

Epoch 2/50
--------------------
train Loss: 0.9584 Acc: 0.7546
valid Loss: 0.7075 Acc: 0.9086

Epoch 3/50
--------------------
train Loss: 0.9445 Acc: 0.7516
valid Loss: 0.7058 Acc: 0.9171

Epoch 4/50
--------------------
train Loss: 0.9351 Acc: 0.7531
valid Loss: 0.7045 Acc: 0.9171

Epoch 5/50
--------------------
train Loss: 0.9462 Acc: 0.7538
valid Loss: 0.6920 Acc: 0.9129

Epoch 6/50
--------------------
train Loss: 0.9230 Acc: 0.7581
valid Loss: 0.6982 Acc: 0.9143

Epoch 7/50
--------------------
train Loss: 0.9414 Acc: 0.7499
valid Loss: 0.6884 Acc: 0.9200

Epoch 8/50
--------------------
train Loss: 0.9081 Acc: 0.7692
valid Loss: 0.6958 Acc: 0.9171

Epoch 9/50
--------------------
train Loss: 0.9232 Acc: 0.7553
valid Loss: 0.6916 Acc: 0.9129

Epoch 10/50
--------------------
train Loss: 0.9091 Acc: 0.7610
valid Loss: 0.7017 Acc: 0.9171

Epoch 11/50
--------------------
train Loss: 0.8

In [ ]:
# Guardamos los pesos del modelo entrenado
torch.save(model_ft.state_dict(), 'resnet18_70_breeds_best_weights_v2.pth')